In [145]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils.extmath import safe_sparse_dot

import numpy as np
import pandas as pd
import math

In [141]:
#Tokyo Race Data
race_data = pd.read_csv("./Downloads/Dataset 0-2.csv",  
                  names=["Distance", "Altitude"])

#Flanders Men's Data
flanders_men_data = pd.read_csv("./Downloads/Flanders Men_s.csv",  
                  names=["Distance", "Altitude"])

#Flanders Women's Data
flanders_women_data = pd.read_csv("./Downloads/Flanders_ Women.csv",  
                  names=["Distance", "Altitude"])

print(race_data)
print(flanders_men_data)
print(flanders_women_data)

     Distance    Altitude
0    0.193820  588.625454
1    0.658223  583.026276
2    1.120367  564.142772
3    1.584751  558.433806
4    2.045234  529.779188
5    2.402284  514.829748
6    2.756840  485.205338
7    3.219040  466.651198
8    3.619187  456.358591
9    4.024291  475.214648
10   4.488097  466.102259
11   4.951567  455.013691
12   5.419181  468.298016
13   5.889820  499.367965
14   6.357004  510.127171
15   6.824208  520.996164
16   7.292681  539.330728
17   7.759978  550.748660
18   8.228003  566.448316
19   8.696943  587.527576
20   9.165864  608.497047
21   9.571514  630.564397
22   9.893525  658.340712
23  10.276548  671.734825
24  10.719911  666.794373
25  11.158923  636.273363
26  11.621403  619.366040
27  12.085209  610.253652
28  12.548585  598.616144
29  13.010673  579.403278
30  13.474909  572.816009
31  13.938434  562.056804
32  14.378761  539.264855
33  14.797745  515.353070
34  15.239715  502.222448
35  15.704622  499.587541
36  16.173711  521.545103
37  16.63606

In [36]:
jackson_data = pd.read_csv("./Downloads/Default Dataset-1.csv",
                          names=["x", "y"])
jackson_data

,x,y
0,11.791667,5.916667
1,12.166667,5.708333
2,12.250000,5.583333
3,13.041667,5.291667
4,12.583333,5.791667
...,...,...
80,7.458333,20.916667
81,7.458333,21.208333
82,7.541667,21.333333
83,7.583333,21.416667


In [341]:
# Polynomial Regression Model (Degree 10)

X = np.array(race_data["Distance"])
y = np.array(race_data["Altitude"])

data = pd.DataFrame.from_dict({
    'x': X,
    'y': y,
})

polyreg = PolynomialFeatures(degree=10).fit_transform(data)

In [340]:
# Polynomial Regression Model for Tokyo
polyreg = make_pipeline(PolynomialFeatures(10), LinearRegression())
polyreg.fit(X.reshape(-1,1),y.reshape(-1,1))
coef = polyreg.named_steps["linearregression"].coef_

# model_1.fit(train_data[model_1_features], train_data['price'])
# print(list(zip(coef, X)))
# print(polyreg.named_steps["polynomialfeatures"].get_feature_names(X.reshape(-1,1).dtype.names))
# print(polyreg[:-1])
# pd.concat([pd.DataFrame(X.reshape(-1,1).dtype.names),pd.DataFrame(np.transpose(polyreg.named_steps["linearregression"].coef_))], axis = 1)

# coefs = pd.DataFrame(
#     polyreg.named_steps["linearregression"].coef_.reshape(11,1),
#     columns=["Coefficients"],
#     index= polyreg[:-1].get_feature_names_out(),
# )

In [125]:
# Gradient Function for Tokyo (x = distance, y = altitude)
def get_altitude(distance):
    sum = 0
    for i in range(11):
        sum += (coef[0][i] * pow(distance, i))
#     arr = np.array([distance])
#     return polyreg.predict(arr.reshape(-1,1))[0][0]
    return sum

#polyreg.predict(np.array([1, 2, 3, 4, 5, 6, 7, 8,9,10,11,12,13,14,15]).reshape(-1,1))
get_altitude(16)

-33.29624335558037

In [303]:
# Define constants
A_m = 1.9 # male surface area
A_f = 1.6 # female surface area
M_m = 88.9 # male mass (kg)
M_f = 71.22 # female mass (kg)
K_a = 0.0108 # air resistance
K_r = 0.143 # rolling resistance
g = 0.00981 # gravitational constant
R = 0.0063 # recovery (s^-1)
MVC = 1000 # watt/newton
k = 0.0153 # fatigue (s^-1)
v_w = 1 # velocity wind
theta_w = 1 # wind direction
mu = 0.75 # coefficient of friction
P_eq = MVC * (R/(2*k))*(-1+(math.sqrt(1+4*(k/R))))

# To populate
theta_rider = [0,20,20,0,20,20]
gradient_rider = [-0.2,0.5,0.8,-0.2,0.5,0.8]
force_arr = [10,250,500,750,1000]
r_rider = [50,4000,200,50,4000,200]
segment_len_arr = [34,53,12,34,53,12] 

In [336]:
# Male

# Quadratic solver to get new velocity given a particular applied force
def get_new_velocity(mass, prev_vel, force, i):
#     a = 0 - (mass/2)
#     b = (1 / prev_vel) + (mass * prev_vel)
#     c = (K_a * A_m * prev_segment_len * pow(prev_vel + (v_w * math.sin(abs(theta_rider[i-1] - theta_w))), 2) 
#     + (g * gradient_rider[i-1] * mass * prev_segment_len) + (K_r * mass * prev_segment_len) 
#     + ((mass * pow(prev_vel,2)) / 2)) / (force * prev_segment_len)
    
    prev_segment_len = segment_len_arr[i-2] if i > 1 else segment_len_arr[0]
    a = (1 / 2) * (mass * prev_vel / prev_segment_len)
    b = 0 - force
    c = (K_a * A_m * prev_vel * pow(prev_vel + (v_w * math.sin((math.pi / 2) - abs(theta_rider[i-1] - theta_w))), 2) 
    - (g * gradient_rider[i-1] * mass * prev_vel) + (K_r * mass * prev_vel) 
    - ((mass * pow(prev_vel,3)) / (2 * prev_segment_len)))
         
    result = (-b + math.sqrt(b**2 - 4*a*c)) / (2 * a)
    return result

# Call first instance of recursion
def dfs():
    final_max = -10000000
    for j in range(5):
        final_max = max(final_max, dfs_helper(force_arr[j], 1000, 10, 0, 0, 0, 1))
    return final_max
    
def dfs_helper(force, prev_f_max, prev_vel, prev_vel_avg, prev_force, r, i):
    if i == len(segment_len_arr):
        maxi = -10000000
        # At last race segment, no more recursive calls. Just return the fastest possible average velocity
        for j in range(5):
            new_f_max = prev_f_max - (k * prev_f_max * prev_force / MVC) + (R * (MVC - prev_f_max))
            new_vel = get_new_velocity(M_m, prev_vel, force_arr[j], i)
         
            if (check_constraints(force, new_f_max, prev_vel, new_vel, segment_len_arr[i-1], i)):
                maxi = max(maxi, (((prev_vel_avg * (i-1)) + new_vel) / i))
        return maxi
    new_f_max = prev_f_max - (k * prev_f_max * prev_force / MVC) + (R * (MVC - prev_f_max))
    max_v_avg = -10000000
    for j in range(5):
        # Calculate new velocity given chosen force
        new_vel = get_new_velocity(M_m, prev_vel, force_arr[j], i)
        print(prev_vel, i, force_arr[j], new_vel)
        
        # If the new velocity and max force are within constraints, run DFS, if not, then go to next iteration.
        if (check_constraints(force_arr[j], new_f_max, prev_vel, new_vel, segment_len_arr[i-1], i)):
            new_vel_avg = new_vel if (i == 0) else ((prev_vel_avg * (i-1)) + new_vel) / i
            max_v_avg = max(max_v_avg, dfs_helper(force_arr[j], new_f_max, new_vel, new_vel_avg, force, r_rider[i], i+1))
    return max_v_avg

# Check for our 4 constraints
def check_constraints(force, new_f_max, prev_vel, new_vel, prev_segment_len, t):
    sof_bool = (new_f_max >= P_eq) and (new_f_max <= MVC)
    r_prev = r_rider[0] if t < 2 else r_rider[t-2]
    velocity_bool = ((force * math.sqrt(mu * g * r_rider[t-1])) - 
                (K_a * A_m * math.sqrt(mu * g * r_rider[t-1]) * pow((math.sqrt(mu * g * r_rider[t-1]) + v_w * 
                math.sin(abs(theta_rider[t-1] - theta_w))),2) - (g * gradient_rider[t-1] * M_m * 
                math.sqrt(mu * g * r_rider[t-1])) + (K_r *  math.sqrt(mu * g * r_rider[t-1]) * M_m) + 
                 (M_m * pow((math.sqrt(mu * g * r_rider[t-1]) - math.sqrt(mu * g * r_prev)),2) 
                  * math.sqrt(mu * g * r_rider[t-1]) / (2 * prev_segment_len)))) <= 0 # constrain r
    return (new_vel <= 45) and (new_vel >= 0) and sof_bool and velocity_bool

In [337]:
dfs()

10 1 10 9.79229161274497
9.79229161274497 2 10 9.60070734301687
9.60070734301687 3 10 9.309436641816156
9.309436641816156 4 10 9.237305884231866
9.237305884231866 5 10 9.043482308771118
9.237305884231866 5 250 23.82049664441736
9.237305884231866 5 500 43.12577729078
9.237305884231866 5 750 63.27861250765141
9.237305884231866 5 1000 83.69371718761589
9.309436641816156 4 250 13.41208493311292
9.309436641816156 4 500 18.87772821364152
9.309436641816156 4 750 25.04891087425688
9.309436641816156 4 1000 31.613559446220656
9.60070734301687 3 250 33.303642406712946
9.60070734301687 3 500 63.28381493363322
9.60070734301687 3 750 93.94500013884887
9.60070734301687 3 1000 124.79588722161122
9.79229161274497 2 250 23.180972665059492
9.79229161274497 2 500 41.1158592676007
9.79229161274497 2 750 59.99601229687379
9.79229161274497 2 1000 79.18226732029716
10 1 250 22.970904323836688
10 1 500 40.43159859753593
10 1 750 58.86943686615468
10 1 1000 77.629170535353
10 1 10 9.79229161274497
9.79229161274

13.63517379530792